In [74]:
import pandas as pd
import math
import numpy as np

In [75]:
# Импорт компосовской херни
import pythoncom
from win32com.client import Dispatch, gencache

import LDefin2D
import MiscellaneousHelpers as MH

In [76]:
#  Подключим константы API Компас
kompas6_constants = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
kompas6_constants_3d = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants

#  Подключим описание интерфейсов API5
kompas6_api5_module = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
kompas_object = kompas6_api5_module.KompasObject(
    Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(kompas6_api5_module.KompasObject.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iKompasObject = kompas_object

#  Подключим описание интерфейсов API7
kompas_api7_module = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
application = kompas_api7_module.IApplication(
    Dispatch("Kompas.Application.7")._oleobj_.QueryInterface(kompas_api7_module.IApplication.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iApplication = application

Documents = application.Documents
#  Получим активный документ
kompas_document = application.ActiveDocument
kompas_document_2d = kompas_api7_module.IKompasDocument2D(kompas_document)
iDocument2D = kompas_object.ActiveDocument2D()

In [77]:
df = pd.read_excel('../../Геодезия.xlsm', sheet_name='pandas')

df

,№№ точек,"Отметки H, м",Unnamed: 2,ПО,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,76.36
0,ПК 0,83.0410,NaN,103.500000,NaN,NaN,91.200000,NaN,NaN,NaN,NaN,79.7005
1,ПК 1,80.4990,NaN,102.553846,NaN,NaN,9.461538,0.1,NaN,NaN,NaN,NaN
2,ПК 2,75.7050,NaN,101.607692,NaN,101.607692,1.892308,0.2,NaN,NaN,NaN,NaN
3,35,73.6470,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ПК 3,73.7580,NaN,100.661538,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN
5,ПК 4,78.3720,NaN,99.715385,NaN,NaN,NaN,0.4,NaN,NaN,NaN,NaN
6,71,80.0195,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ПК 5,79.5170,NaN,98.769231,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN
8,ПК 6,77.0150,NaN,97.823077,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN
9,ПК 7,72.2140,NaN,96.876923,NaN,NaN,NaN,0.7,NaN,NaN,NaN,NaN


In [78]:
HORIZONTAL_TEXT_AXIS = [i - (84.814180186008 - 84.421252) for i in [84.814180186008, 94.814180186008, 104.814180186008, 108.314180186008, 114.814180186008,
                        124.814180186008, 131.914180186008, 134.814180186008, 144.814180186008, 154.814180186008,
                        164.814180186008, 167.714180186008, 174.814180186008, 184.814180186008, 192.314180186008,
                        194.814180186008, 204.814180186008, 214.814180186008, 224.814180186008, 229.814180186008,
                        234.814180186008, 239.714180186008, 244.814180186008, 254.814180186008, 264.814180186008,
                        274.814180186008, 282.014180186008, 284.814180186008,] ]

POINT_NAMES = [0,1,2,35,3,4,71,5,6,7,8,29,9,10,75,11,12,13,14,50,15,49,16,17,18,19,72,20,]

BASELINE = 137.703368

Y_MIN = math.floor(min(df["Отметки H, м"])) - 2

In [79]:
heights = [[HORIZONTAL_TEXT_AXIS[i], df["Отметки H, м"][i], df["ПО"][i]] for i in range(len(POINT_NAMES))]

In [80]:
def add_layer(number: int, state: int = 3, name: str = None) -> None:
    obj = iDocument2D.ksLayer(number)
    i_layer_param = kompas6_api5_module.ksLayerParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_LayerParam))
    i_layer_param.Init()
    if name:
        i_layer_param.name = name
    i_layer_param.state = state
    iDocument2D.ksSetObjParam(obj, i_layer_param, LDefin2D.ALLPARAM)


def add_text(text: str, x: float, y: float, angle: float = 0, font_size: float = 2.5, color=0) -> None:
    i_paragraph_param = kompas6_api5_module.ksParagraphParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_ParagraphParam))
    i_paragraph_param.Init()
    i_paragraph_param.x = x
    i_paragraph_param.y = y
    i_paragraph_param.ang = angle
    i_paragraph_param.height = 3.55
    i_paragraph_param.width = 4
    i_paragraph_param.hFormat = 0
    i_paragraph_param.vFormat = 0
    i_paragraph_param.style = 1
    iDocument2D.ksParagraph(i_paragraph_param)

    i_text_line_param = kompas6_api5_module.ksTextLineParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextLineParam))
    i_text_line_param.Init()
    i_text_line_param.style = 1
    i_text_item_array = kompas_object.GetDynamicArray(LDefin2D.TEXT_ITEM_ARR)
    i_text_item_param = kompas6_api5_module.ksTextItemParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextItemParam))
    i_text_item_param.Init()
    i_text_item_param.iSNumb = 0
    i_text_item_param.s = text
    i_text_item_param.type = 0
    i_text_item_font = kompas6_api5_module.ksTextItemFont(i_text_item_param.GetItemFont())
    i_text_item_font.Init()
    i_text_item_font.bitVector = 4096
    i_text_item_font.color = color
    i_text_item_font.fontName = "GOST type A"
    i_text_item_font.height = font_size
    i_text_item_font.ksu = 1
    i_text_item_array.ksAddArrayItem(-1, i_text_item_param)
    i_text_line_param.SetTextItemArr(i_text_item_array)

    iDocument2D.ksTextLine(i_text_line_param)
    iDocument2D.ksEndObj()



In [81]:
if input("Добавить вертикальную текстовую ось (y/n)? \n") == "y":
    add_layer(1100, 3, "Текст Ось Y")
    x = 79
    y = 146.5
    n = Y_MIN + 2
    for i in range(1, 17):
        add_text(n, x, y)
        n += 2
        y += 10

In [82]:
if input("Добавить 'Отметки земли, м' (y/n)? \n") == "y":
    add_layer(1110, 3, "Отметки земли, м")
    for point in heights:
        add_text("{:1.3f}".format(point[1]), point[0] + 1.2, 70.2, 90)

In [83]:
if input("Добавить 'Проектные отметки, м' (y/n)? \n") == "y":
    add_layer(1110, 3, "Проектные отметки, м")
    for point in heights:
        add_text("{:1.3f}".format(point[2]), point[0] + 1.2, 95.2, 90)

In [84]:
if input("Рисовать вертикали? (y/n) \n") == "y":
    add_layer(1120, 3, "Вертикали")
    for point in heights:
        obj = iDocument2D.ksLineSeg(point[0], BASELINE, point[0],BASELINE + (point[1] - Y_MIN)*5, 2)
    if input("Соединять вертикали? (y/n) \n") == "y":
        for i in range(len(heights)):
                if i == 0:
                    continue
                obj = iDocument2D.ksLineSeg(heights[i - 1][0], BASELINE + (heights[i - 1][1] - Y_MIN) * 5, heights[i][0], BASELINE + (heights[i][1] - Y_MIN) * 5, 2)


In [85]:
print('Done!')

Done!


In [86]:
if input("Добавить 'Отметки земли, м' (y/n)? \n") == "y":
    add_layer(1110, 3, "Отметки земли, м")
    for point in heights:
        add_text("{:1.3f}".format(point[1]), point[0] + 1.2, 70.2, 90)

In [87]:
_slopes = [(204.5, 89), (275.5, 89)]